# Notebook to design trading bot

Latest version: 2024-08-14  
Author: MvS

## Description

Example of a trading bot using the RSI indicator to decide on trades

## Result




In [ ]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from lumibot.entities import TradingFee, Asset

from alpaca_trade_api import REST
import pandas as pd

from datetime import datetime, timedelta
from dotenv import dotenv_values

# from utils.finbert_utils import estimate_sentiment

env_dict = dotenv_values("../.env")

In [ ]:
ALPACA_CREDS = {
    "BASE_URL": env_dict['ALP_API_URL'],
    "API_KEY": env_dict['ALP_API_KEY'],
    "API_SECRET": env_dict['ALP_API_SECRET'],
    "PAPER": True,
}

ALPACA_CREDS

In [ ]:
# Define the MyBot strategy class, inheriting from the Strategy base class
class MyBot(Strategy):
    def initialize(self):
        # Initialize strategy parameters
        self.symbol = self.parameters["symbol"]
        self.sleeptime = self.parameters["sleeptime"]
        self.cash_at_risk = self.parameters["cash_at_risk"]
        self.rsi_period = self.parameters["rsi_period"]
        self.overbought = self.parameters["overbought"]
        self.oversold = self.parameters["oversold"]
        self.api = REST(
            base_url=self.parameters["credentials"]["BASE_URL"],
            key_id=self.parameters["credentials"]["API_KEY"],
            secret_key=self.parameters["credentials"]["API_SECRET"],
        )  # Initialize Alpaca REST API
        self.data = (
            pd.DataFrame()
        )  # Initialize an empty DataFrame for storing price data
        self.last_trade = None  # Track the last trade action (buy or sell)

    def calculate_rsi(self, data):
        """
        Calculate the Relative Strength Index (RSI) based on the historical price data.

        :param data: DataFrame containing historical price data
        :return: Series containing the RSI values
        """
        delta = data[
            'close'
        ].diff()  # Calculate the difference between consecutive close prices
        gain = (
            (delta.where(delta > 0, 0)).rolling(window=self.rsi_period).mean()
        )  # Calculate rolling average of gains
        loss = (
            (-delta.where(delta < 0, 0)).rolling(window=self.rsi_period).mean()
        )  # Calculate rolling average of losses
        rs = gain / loss  # Calculate the Relative Strength (RS)
        rsi = 100 - (100 / (1 + rs))  # Calculate the RSI using the RS
        return rsi

    def update_data(self):
        """
        Fetch and update the historical price data for the specified symbol.
        """
        end_date: datetime = (
            self.get_datetime()
        )  # Get the current date and time in the backtest or live environment

        # Adjust start_date to fetch sufficient data for RSI calculation
        start_date = end_date - timedelta(days=self.rsi_period * 2)

        # Fetch historical price data from Alpaca
        bars = self.api.get_bars(
            symbol=self.symbol,
            timeframe='1D',
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d'),
        )

        # Reformatting the data into a DataFrame
        data = []
        for bar in bars:
            data.append(
                {
                    'timestamp': bar.t,
                    'open': bar.o,
                    'high': bar.h,
                    'low': bar.l,
                    'close': bar.c,
                    'volume': bar.v,
                }
            )
        self.data = pd.DataFrame(data)  # Create a DataFrame from the fetched data
        self.data.set_index('timestamp', inplace=True)  # Set timestamp as the index
        self.data.index = pd.to_datetime(
            self.data.index
        )  # Ensure the index is in datetime format
        self.data['RSI'] = self.calculate_rsi(
            self.data
        )  # Calculate and store RSI values

    def check_rsi_signal(self):
        """
        Check for RSI buy/sell signals based on current market conditions.

        :return: 'buy' if RSI indicates an oversold condition, 'sell' if overbought, or None if no signal.
        """
        self.update_data()  # Update the historical data
        rsi = self.data['RSI'].iloc[-1]  # Get the most recent RSI value
        previous_rsi = self.data['RSI'].iloc[-2]  # Get the previous RSI value

        # Generate a buy signal if the RSI has just crossed below the oversold threshold
        if rsi < self.oversold and previous_rsi >= self.oversold:
            return 'buy'
        # Generate a sell signal if the RSI has just crossed above the overbought threshold
        elif rsi > self.overbought and previous_rsi <= self.overbought:
            return 'sell'
        return None  # Return None if no signal is generated

    def on_trading_iteration(self):
        """
        Execute trading logic on each iteration, based on RSI signals and current cash position.
        """
        last_price = self.get_last_price(self.symbol)  # Get the last price of the asset
        quantity = round(
            self.cash * self.cash_at_risk / last_price or 0, 0
        )  # Calculate the quantity to buy based on available cash
        position = (
            self.get_position(self.symbol) or 0
        )  # Get the current position (if any)
        signal = self.check_rsi_signal()  # Check for buy/sell signals based on RSI

        if (
            self.cash > last_price and signal
        ):  # Ensure enough cash is available and a signal is present
            if signal == "buy":
                if self.last_trade == "sell":
                    self.sell_all()  # Sell all if the last trade was a sell
                order = self.create_order(
                    asset=self.symbol,
                    quantity=quantity,
                    side="buy",
                )
                self.submit_order(order)  # Submit a buy order
                self.last_trade = "buy"  # Update last trade to buy
            elif signal == "sell":
                if self.last_trade == "buy":
                    self.sell_all()  # Sell all if the last trade was a buy
                order = self.create_order(
                    asset=self.symbol,
                    quantity=quantity,
                    side="sell",
                )
                self.last_trade = "sell"  # Update last trade to sell

In [ ]:
benchmark_symbol = "AAPL"

# Set strategy parameters
parameters = {
    "symbol": 'AAPL',
    "cash_at_risk": 0.5,
    "sleeptime": '24H',
    "rsi_period": 14,
    "overbought": 70,
    "oversold": 30,
    "benchmark_asset": Asset(symbol=benchmark_symbol, asset_type="stock"),
    "credentials": ALPACA_CREDS,
}

# Define the backtesting parameters

end_date = datetime.today() - timedelta(days=30)
# Define real-time interval:
#  - assume to display at least the number of sample points of the larger period
#  - this requires double the number of points to create the averaging
#  - plus considering non-trading days - yfinance returns only trading days, howevers
start_date = end_date - timedelta(days=365)
start_date = datetime(2023, 1, 1)

broker = Alpaca(ALPACA_CREDS)  # Initialize Alpaca broker with credentials
strategy = MyBot(
    name='Forexbot',
    broker=broker,
    parameters=parameters,
    force_start_immediately=True,
)

# Create trading fees for backtesting

# Set a flat $5 trading fee for each trade
trading_fee = TradingFee(flat_fee=5)

# Run backtesting
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    benchmark_asset=benchmark_symbol,
    budget=1000,
    buy_trading_fees=[trading_fee],
    sell_trading_fees=[trading_fee],
    parameters=parameters,
)

# Uncomment the following lines to run the strategy on a paper trading account.

# trader = Trader()
# trader.add_strategy(strategy)
# trader.run_all()